# INIT

START

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

import ipytest
ipytest.autoconfig()
from time import sleep

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string, base64_to_hex
from utils.utils_chain import nominated_amount

from contracts.farm_contract import FarmContract
from utils.contract_retrievers import retrieve_farm_by_address

from multiversx_sdk import ProxyNetworkProvider, ApiNetworkProvider

SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"
GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api

In [ ]:
farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_index(config.FARMS_V2, 0)
print(f"Using {farm_contract.address} : {farm_contract.farmToken}")

CHAIN CONFIG SETUP

In [23]:
import json
from typing import Any
from utils.utils_chain import WrapperAddress

USERS = ["erd1ndyxz4gpfncmz82qm39yqramgd826mkalhqw8tfmmy0g9jecqc5sa20vek", # user with both farm and farming positions
         "erd1gqyspww4pssv6ck6pl8vtnl9tnwe9hy5d2324mya4rz5ma9dtp9snjgd7f", # user with only farm position
         "erd15gzp9k56cnn8qtfxwlghcxgs74v8jmfk4crex9alygxldmpg9f5s3fd4pl",] # user with old farm position

def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    

def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", state)


def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def setup_chain_sim():
    # generate blocks to pass an epoch and the smart contract deploys to be enabled
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/5", {})

    load_and_apply_state(proxy, PROJECT_ROOT,
                         context.deployer_account.address.bech32(),
                         USERS)


def advance_blocks(number_of_blocks: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})


def users_init() -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in USERS:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

Farm upgrade

In [6]:
def farm_upgrade():
    tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                            "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/farm-with-locked-rewards.wasm", 
                                            [], True)

    advance_blocks(1)
    tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)
    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
    assert base64_to_hex(code_hash) == "4ca038c097412353ec6cf6039f2a216aa70725d80d2a647b7948bc466d0d4444"

Deploy permissions hub

In [7]:
from contracts.permissions_hub_contract import PermissionsHubContract

def deploy_permissions_hub():
    permissions_hub_contract = PermissionsHubContract("")
    _, address = permissions_hub_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                            "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/permissions-hub.wasm",
                                            [])
    permissions_hub_contract.address = address
    return permissions_hub_contract

Dummy proxy

In [8]:
from contracts.dummy_proxy_contract import DummyProxyContract

def deploy_dummy_proxy_contract():
    dummy_proxy_contract = DummyProxyContract("")
    _, address = dummy_proxy_contract.contract_deploy(context.deployer_account, context.network_provider.proxy,
                                         "https://github.com/ovidiuolteanu/mx-sc-dummy-proxy/releases/download/v2.1/dummy-proxy.wasm",
                                         [])
    dummy_proxy_contract.address = address
    return dummy_proxy_contract

In [9]:
from utils.utils_chain import get_all_token_nonces_details_for_account

def get_position_for_account(user_address: str):
    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    return farm_tk_nonce, farm_tk_balance

def get_farming_token_balance_for_account(user_address: str):
    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmingToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")
    
    return farming_tk_balance

In [10]:
from utils.utils_tx import ESDTToken, multi_esdt_transfer

def send_tokens_to_dummy(user: Account, dummy_contract: DummyProxyContract):
    farm_tk_nonce, farm_tk_balance = get_position_for_account(user)
    print(f"Sending {farm_tk_balance} {farm_contract.farmToken}-{farm_tk_nonce} to dummy contract")
    multi_esdt_transfer(context.network_provider.proxy, 20000000, user, dummy_contract.address, [ESDTToken(farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)])

Claim

In [11]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

    tx_hash = farm_contract.claimRewards(context.network_provider, user_account, event)
    return tx_hash

Claim boosted

In [12]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_boosted_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)
    event = ClaimRewardsFarmEvent(0, 0, '', user_account.address.bech32())
    tx_hash = farm_contract.claim_boosted_rewards(context.network_provider, user_account, event)
    return tx_hash

Claim on behalf

In [13]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_on_behalf_from_user(claim_account: Account):
    claim_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, claim_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

    tx_hash = farm_contract.claim_rewards_on_behalf(context.network_provider, claim_account, event)
    return tx_hash

Enter farm consolidated

In [14]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)

    tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event)
    return tx_hash

Enter farm no consolidation

In [15]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_no_consolidation_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                       "", 0, 0)

    tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event, 0, True)
    return tx_hash

Enter farm on behalf

In [16]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_on_behalf_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance, user_account.address.bech32())

    tx_hash = farm_contract.enter_farm_on_behalf(context.network_provider, caller_account, event)
    return tx_hash

Enter farm on behalf no consolidation

In [17]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_on_behalf_no_consolidation_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                           "", 0, 0, user_account.address.bech32())

    tx_hash = farm_contract.enter_farm_on_behalf(context.network_provider, caller_account, event)
    return tx_hash

In [18]:
def dict_compare(d1, d2):
    print(d1)
    print(d2)
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    added = d1_keys - d2_keys
    removed = d2_keys - d1_keys
    modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
    same = set(o for o in shared_keys if d1[o] == d2[o])
    return added, removed, modified, same

def check_equal_dicts(dict1, dict2):
    """
    Compare two dictionaries, including nested dictionaries.
    
    Args:
    dict1 (dict): First dictionary to compare.
    dict2 (dict): Second dictionary to compare.
    
    Returns:
    bool: True if dictionaries are equal, False otherwise.
    """
    if dict1.keys() != dict2.keys():
        return False
    
    for key in dict1:
        if isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
            if not check_equal_dicts(dict1[key], dict2[key]):
                return False
        elif dict1[key] != dict2[key]:
            return False
    
    return True

In [19]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, WrapperAddress, get_all_token_nonces_details_for_account

def user_farm_token_stats(user):
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.address.bech32(), context.network_provider.proxy)
    print(f'Account: {user.address.bech32()}')
    print(f'Looking for {farm_contract.farmingToken} and {farm_contract.farmToken} tokens')
    print(f'Farming Tokens in account:')
    for token in tokens_in_account:
        print(f'\t{token}')
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
    print(f'Farm Tokens in account:')
    all_decoded_attributes = []
    for token in tokens_in_account:
        print(f'\t{token}')
        decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)
        print(f'\t\t{decoded_attributes}')
        all_decoded_attributes.append(decoded_attributes)
        
    return all_decoded_attributes

CHAIN SIMULATOR STACK

In [20]:
import subprocess
from time import sleep

CS_DOCKER_PATH = Path.home() / "projects/testing/full-stack-docker-compose/chain-simulator"

def start_chain_sim_stack():
    # stop first in case one is already running
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    
    p = subprocess.Popen(["docker", "compose", "up", "-d"], cwd = CS_DOCKER_PATH)
    sleep(60)
    return p

def stop_chain_sim_stack(p):
    p.terminate()
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    _ = subprocess.run(["docker", "system", "prune", "-f"], cwd = CS_DOCKER_PATH)

# OPERATIONS

In [ ]:
setup_chain_sim()

ADVANCE BLOCKS

In [ ]:
proxy.do_post(f"{GENERATE_BLOCKS_URL}/1", {})

In [ ]:
proxy.do_post(f"{GENERATE_BLOCKS_URL}/3", {})

User setup

In [ ]:
users_init()

USER TOKEN STATS

In [ ]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, WrapperAddress, get_all_token_nonces_details_for_account

# user = Account(pem_file=config.DEFAULT_ACCOUNTS)
# user.address = WrapperAddress("")
# user.sync_nonce(context.network_provider.proxy)

tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.address.bech32(), context.network_provider.proxy)
print(f'Account: {user.address.bech32()}')
print(f'Looking for {farm_contract.farmingToken} and {farm_contract.farmToken} tokens')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

In [ ]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, WrapperAddress, get_all_token_nonces_details_for_account

# user = Account(pem_file=config.DEFAULT_ACCOUNTS)
# user.address = WrapperAddress("")
# user.sync_nonce(context.network_provider.proxy)

tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.address.bech32(), context.network_provider.proxy)
print(f'Account: {user.address.bech32()}')
print(f'Looking for {farm_contract.farmingToken} and {farm_contract.farmToken} tokens')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Get user & farm stats

In [ ]:
import json
print(json.dumps(farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy), indent=4))
print(json.dumps(farm_contract.get_all_stats(context.network_provider.proxy), indent=4))

In [ ]:
import json
print(json.dumps(farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy), indent=4))
print(json.dumps(farm_contract.get_all_stats(context.network_provider.proxy), indent=4))

upgrade contract

In [ ]:
# pair_address = farm_contract.get_lp_address(context.network_provider.proxy)
tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/farm-with-locked-rewards.wasm", 
                                         [], True)

resume contract

In [ ]:
farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Basic user

In [ ]:
from utils.utils_chain import WrapperAddress
user = Account(pem_file=config.DEFAULT_ADMIN)
user.address = WrapperAddress("erd13dj5qy3ht8pewf7vme50gle2d2lx0wzywa2cvez08x5zzu5x6d7sewjt3m")
user.sync_nonce(context.network_provider.proxy)

Collect user accounts

In [ ]:
from utils.utils_scenarios import collect_farm_contract_users
from multiversx_sdk import ApiNetworkProvider

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
fetched_users = collect_farm_contract_users(20, farm_contract.address, farm_contract.farmingToken, farm_contract.farmToken,
                                            mainnet_api, context.network_provider.proxy)

Used users

In [ ]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, get_all_token_nonces_details_for_account
from utils.utils_scenarios import FetchedUser
from typing import List

index = 0
users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
if not users:
    raise Exception('No users found for the given criteria')

user = users[index].address
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.bech32(), context.network_provider.proxy)
print(f'Account: {user.bech32()}')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Find user by token

In [ ]:
from multiversx_sdk.network_providers import ApiNetworkProvider
from utils.utils_chain import dec_to_padded_hex, WrapperAddress, get_all_token_nonces_details_for_account, decode_merged_attributes, base64_to_hex
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES

migration_nonce = 152776

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
for nonce in reversed(range(migration_nonce)):
    url = f'nfts/{farm_contract.farmToken}-{dec_to_padded_hex(nonce)}/accounts'
    try:
        response = mainnet_api.do_get_generic(url)
        user_found = ""
        for entry in response:
            if WrapperAddress(entry['address']).is_smart_contract():
                continue
            user_found = entry['address']
        
        if user_found:
            tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_found, context.network_provider.proxy)
            for token in tokens_in_account:
                token_decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)
                print(f'Farm Tokens nonce {dec_to_padded_hex(nonce)}: {token_decoded_attributes}')
    except Exception as e:
        print(f'Error fetching {url}: {e}')
        continue

In [ ]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, WrapperAddress, get_all_token_nonces_details_for_account

# user = Account(pem_file=config.DEFAULT_ACCOUNTS)
# user.address = WrapperAddress("")
# user.sync_nonce(context.network_provider.proxy)

tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.address.bech32(), context.network_provider.proxy)
print(f'Account: {user.address.bech32()}')
print(f'Looking for {farm_contract.farmingToken} and {farm_contract.farmToken} tokens')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Transfer position

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken
from utils.utils_chain import Account, WrapperAddress
fetched_user = users[0]
user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
user_account.address = fetched_user.address
user_account.sync_nonce(context.network_provider.proxy)

destination = WrapperAddress("")

full_token_id = fetched_user.farm_tokens[0].get('tokenIdentifier')
token_id = '-'.join(full_token_id.split('-')[:-1])
token = ESDTToken(token_id, fetched_user.farm_tokens[0].get('nonce'), int(fetched_user.farm_tokens[0].get('balance'))//2)
print(f'Sending: {token.get_token_data()} from {fetched_user.address.bech32()}')
multi_esdt_transfer(context.network_provider.proxy, 1000000, user_account, destination, [token])

Enter farm

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address

# user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
# user_account.address = user
# user_account.sync_nonce(context.network_provider.proxy)

user_account = user

farming_tk_balance = 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farming tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > 0:
        farming_tk_balance = int(token['balance'])
        break

if not farming_tk_balance:
    raise Exception("Not enough farming token balance")

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farm tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

# use only 10% of the balance
event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance // 10,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)

initial_enter: bool = False if farm_tk_nonce else True
tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event, initial=initial_enter)

In [ ]:
proxy_contract.set_transfer_role_wrapped_lp_token(context.deployer_account, context.network_provider.proxy, proxy_contract.address)

Claim rewards

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user.address
user_account.sync_nonce(context.network_provider.proxy)

event = ClaimRewardsFarmEvent(0, 0, '', user_account.address.bech32())

tx_hash = farm_contract.claim_boosted_rewards(context.network_provider, user_account, event)

Permissions hub setup

In [ ]:
from contracts.permissions_hub_contract import PermissionsHubContract

permissions_hub_contract: PermissionsHubContract
permissions_hub_contract = context.get_contracts(config.PERMISSIONS_HUBS)[0]
print(permissions_hub_contract.address)

Claim on behalf

In [ ]:
permissions_hub_contract.add_to_whitelist(user_account, context.network_provider.proxy, ["erd1rf4hv70arudgzus0ymnnsnc4pml0jkywg2xjvzslg0mz4nn2tg7q7k0t6p"])

Get user farm position

In [ ]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

Exit rewards

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ExitFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = ExitFarmEvent(farm_contract.farmToken, farm_tk_balance, farm_tk_nonce, '')

tx_hash = farm_contract.exitFarm(context.network_provider, user_account, event)

Get user farm position

In [ ]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user_account.address.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

SHADOWFORK CONTROL

In [ ]:
from contracts.builtin_contracts import SFControlContract

epocs_to_advance = 7

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')
context.deployer_account.sync_nonce(context.network_provider.proxy)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epocs_to_advance, 9)

wait for epoch fast forward

In [ ]:
from time import sleep

target_epoch = current_epoch + epocs_to_advance
while current_epoch < target_epoch:
    current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
    print(f'Current epoch: {current_epoch}')
    sleep(10)
print('Reached target epoch!')

Trackers

In [ ]:
from trackers.farm_economics_tracking import FarmEconomics
from contracts.farm_contract import FarmContractVersion

farm_tracker = FarmEconomics(farm_contract.address, FarmContractVersion.V2Boosted, context.network_provider)

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()

State

In [ ]:
current_week = farm_contract.get_current_week(context.network_provider.proxy)
print(current_week)

In [ ]:
energy_for_user_in_week = farm_contract.get_user_energy_for_week(user.bech32(), proxy=context.network_provider.proxy, week=current_week)
print(energy_for_user_in_week)

In [ ]:
user_progress = farm_contract.get_current_claim_progress_for_user(user.bech32(), context.network_provider.proxy)
print(user_progress)

In [ ]:
last_week = farm_contract.get_last_active_week_for_user(user.bech32(), context.network_provider.proxy)
print(last_week)

In [ ]:
from tools.runners.farm_runner import get_farm_addresses_from_chain
from contracts.farm_contract import FarmContract

farm_addresses = get_farm_addresses_from_chain("v2")

for farm_address in farm_addresses:
    farm_contract = FarmContract.load_contract_by_address(farm_address)
    stats = farm_contract.get_all_stats(context.network_provider.proxy)

    print(f'Farm: {farm_address}')
    print(f'Farm identity: {farm_contract.get_config_dict()}')
    print(f'Stats: {stats}')
    print('')


New Farm setup

In [ ]:
FARM_TOKEN_NAME = ""
FARM_TOKEN_TICKER = ""
POOL_ADDRESS = ""
LOCK_EPOCHS = 1440
BOOSTED_YIELD_REWARDS = 6000
BOOSTED_YIELD_FACTORS = [2, 1, 0, 1, 1]
REWARDS_PER_BLOCK = 1
PENALTY_PERCENT = 300
MINIMUM_FARMING_EPOCH = 7

In [ ]:
from contracts.farm_contract import FarmContract, FarmContractVersion
from contracts.pair_contract import PairContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

pair_contract = PairContract.load_contract_by_address(POOL_ADDRESS)
print(f'Pair contract: {pair_contract.address} {pair_contract.lpToken}')
energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

farm_contract: FarmContract
farm_contract = FarmContract(pair_contract.lpToken, "", energy_factory.base_token, "", FarmContractVersion.V2Boosted)

In [ ]:
hash, address = farm_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                              config.FARM_V3_BYTECODE_PATH,
                                              [POOL_ADDRESS, context.admin_account.address.bech32()])
farm_contract.address = address

In [ ]:
farm_contract.register_farm_token(context.deployer_account, context.network_provider.proxy, [FARM_TOKEN_NAME, FARM_TOKEN_TICKER])

In [ ]:
pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, farm_contract.address)

In [ ]:
farm_contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, energy_factory.address)

In [ ]:
farm_contract.set_locking_address(context.deployer_account, context.network_provider.proxy, energy_factory.address)

In [ ]:
energy_factory.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, farm_contract.address)

In [ ]:
farm_contract.set_lock_epochs(context.deployer_account, context.network_provider.proxy, LOCK_EPOCHS)

In [ ]:
farm_contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, BOOSTED_YIELD_REWARDS)

In [ ]:
farm_contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, BOOSTED_YIELD_FACTORS)

In [ ]:
farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 1)

In [ ]:
farm_contract.set_penalty_percent(context.deployer_account, context.network_provider.proxy, 300)

In [ ]:
farm_contract.set_minimum_farming_epochs(context.deployer_account, context.network_provider.proxy, 7)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [farm_contract.address])

# SCENARIOS

Chain sim control

In [ ]:
chain_sim_stack = start_chain_sim_stack()

In [ ]:
stop_chain_sim_stack(chain_sim_stack)

Init

In [ ]:
setup_chain_sim()
users = users_init()
advance_blocks(1)

In [ ]:
user_farm_token_stats(user)

Claim for user

In [ ]:
claim_for_user(user)
advance_blocks(1)

In [ ]:
user = users_init()[0]
claim_on_behalf_from_user(user)
advance_blocks(1)

In [10]:
advance_blocks(1)

Claim on behalf

In [ ]:
user = users_init()[0]
permissions_hub_contract = deploy_permissions_hub()
farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)

claim_on_behalf_from_user(user)
advance_blocks(1)

Claim compare

In [ ]:
setup_chain_sim()
user = users_init()[0]
advance_blocks(3)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = claim_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restart Chain Simulator then press Enter to continue...")

setup_chain_sim()
user = users_init()[0]
advance_blocks(1)

farm_upgrade()  # eats 2 blocks

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = claim_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)

u4 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(user)

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u4)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")
print(claim_ops_2)
print(claim_ops_4)

Claim boosted compare

In [ ]:
setup_chain_sim()
user = users_init()[0]
advance_blocks(3)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = claim_boosted_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_boosted_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restart Chain Simulator then press Enter to continue...")

setup_chain_sim()
user = users_init()[0]
advance_blocks(1)

farm_upgrade()  # eats 2 blocks

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = claim_boosted_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_boosted_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash, True)

u4 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(user)

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u4)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")
print(claim_ops_2)
print(claim_ops_4)

Claim on Behalf compare

In [ ]:
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[2]
initial_blocks = 10
advance_blocks(initial_blocks)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = claim_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restarting Chain Simulator. Continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[2]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
permissions_hub_contract = deploy_permissions_hub() # eats 1 block
consumed_blocks += 1

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)
consumed_blocks += 1

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
consumed_blocks += 1


permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)
consumed_blocks += 1

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
if user_tk_nonce:
    token = ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)
    tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.address, "claimRewardsOnBehalf"])
else:
    user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
    tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)

u5 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u6 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u5)
dict_compare(u4, u6)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")
print(claim_ops_2)
print(claim_ops_4)

input("Closing Chain Simulator. Continue...")

stop_chain_sim_stack(chain_sim)

Partial claim on behalf compare

In [ ]:
from utils.utils_tx import multi_esdt_endpoint_call, ESDTToken

chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[2]
initial_blocks = 10
advance_blocks(initial_blocks)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

user.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

# claim half of owned farm tokens
first_claim_balance = farm_tk_balance // 2
event = ClaimRewardsFarmEvent(first_claim_balance, farm_tk_nonce, '')
tx_hash = farm_contract.claimRewards(context.network_provider, user, event)
advance_blocks(1)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)

# claim the rest of the farm tokens
second_claim_balance = farm_tk_balance - first_claim_balance
event = ClaimRewardsFarmEvent(second_claim_balance, farm_tk_nonce, '')
tx_hash = farm_contract.claimRewards(context.network_provider, user, event)
advance_blocks(4)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

# claim all tokens again - should lead to compounding and rewards given only for first position
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
token_payments = []
for token in tokens_in_account:
    token_payments.append(ESDTToken(farm_contract.farmToken, token['nonce'], int(token['balance'])))
gas_limit = 50000000
function = "claimRewards"
sc_args = [
    token_payments
]
tx_hash = multi_esdt_endpoint_call(function, context.network_provider.proxy, gas_limit, user,
                                Address(farm_contract.address), function, sc_args)
advance_blocks(1)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

# claim final position
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_bf = context.network_provider.get_tx_operations(tx_hash, True)

input("Restarting Chain Simulator. Continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[2]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
permissions_hub_contract = deploy_permissions_hub() # eats 1 block
consumed_blocks += 1

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)
consumed_blocks += 1

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
consumed_blocks += 1

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)
consumed_blocks += 1

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())

# claim half of owned farm tokens
first_claim_balance = user_tk_balance // 2
token = ESDTToken(farm_contract.farmToken, user_tk_nonce, first_claim_balance)
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash, True)

# claim the rest of the farm tokens
second_claim_balance = user_tk_balance - first_claim_balance
token = ESDTToken(farm_contract.farmToken, user_tk_nonce, second_claim_balance)
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_5 = context.network_provider.get_tx_operations(tx_hash, True)

# send one of the positions back to the user (limitation of the dummy proxy contract)
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, contract_tk_nonce, contract_tk_balance, user.address, ""])
advance_blocks(3)
sleep(2)

# claim all tokens again via hybrid transfer - should lead to compounding and rewards given only for first position
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
token = ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)
tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.farmToken, contract_tk_nonce, contract_tk_balance, 
                                                                                                        farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_6 = context.network_provider.get_tx_operations(tx_hash)

u5 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u6 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))

# claim final position
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, contract_tk_nonce, contract_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_af = context.network_provider.get_tx_operations(tx_hash)

print(check_equal_dicts(u1, u3))
print(check_equal_dicts(u2, u5))
print(check_equal_dicts(u4, u6))
print(check_equal_dicts(c1, c3))
print(check_equal_dicts(c2, c4))
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_4)
print("-------------------")
print(claim_ops_2)
print(claim_ops_5)
print("-------------------")
print(claim_ops_3)
print(claim_ops_6)
print("-------------------")
print(claim_ops_bf)
print(claim_ops_af)

input("Closing Chain Simulator. Continue...")

stop_chain_sim_stack(chain_sim)

Mixed claim on behalf

In [ ]:
from utils.utils_tx import multi_esdt_endpoint_call, multi_esdt_transfer, ESDTToken

chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
user2 = users_init()[1]
initial_blocks = 10
advance_blocks(initial_blocks)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

user.sync_nonce(context.network_provider.proxy)
user2.sync_nonce(context.network_provider.proxy)

# send user2's position to user
user_tk_nonce, user_tk_balance = get_position_for_account(user2.address.bech32())
multi_esdt_transfer(context.network_provider.proxy, 10000000, user2, user.address, [ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)])
advance_blocks(1)
sleep(2)

# claim all tokens - should lead to compounding and rewards given only for first position
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
token_payments = []
for token in tokens_in_account:
    token_payments.append(ESDTToken(farm_contract.farmToken, token['nonce'], int(token['balance'])))
gas_limit = 50000000
function = "claimRewards"
sc_args = [
    token_payments
]
# TODO: reverse the order of the payments
tx_hash = multi_esdt_endpoint_call(function, context.network_provider.proxy, gas_limit, user,
                                Address(farm_contract.address), function, sc_args)
advance_blocks(1)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

# claim final position
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_bf = context.network_provider.get_tx_operations(tx_hash, True)

input("Restarting Chain Simulator. Continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
user2 = users_init()[1]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
permissions_hub_contract = deploy_permissions_hub() # eats 1 block
consumed_blocks += 1

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)
consumed_blocks += 1

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
consumed_blocks += 1

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)
consumed_blocks += 1

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u32 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

# TODO: reverse the order of the payments - send user's position to dummy, send user2's position to user then hybrid transfer
# send user2's position to user
user_tk_nonce, user_tk_balance = get_position_for_account(user2.address.bech32())
multi_esdt_transfer(context.network_provider.proxy, 10000000, user2, WrapperAddress(dummy_proxy_contract.address), [ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)])
advance_blocks(1)
sleep(2)

# claim all tokens via hybrid transfer - should lead to compounding and rewards given only for first position
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
token = ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)
tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.farmToken, contract_tk_nonce, contract_tk_balance, 
                                                                                                        farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_6 = context.network_provider.get_tx_operations(tx_hash)

u5 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u52 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
u6 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))

# claim final position
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, contract_tk_nonce, contract_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_af = context.network_provider.get_tx_operations(tx_hash)

print(f'{u1=}\n{u3=}\n{check_equal_dicts(u1, u3)}')
print(f'{u12=}\n{u32=}\n{check_equal_dicts(u12, u32)}')
print(f'{u2=}\n{u5=}\n{check_equal_dicts(u2, u5)}')
print(f'{u22=}\n{u52=}\n{check_equal_dicts(u22, u52)}')
print(f'{u4=}\n{u6=}\n{check_equal_dicts(u4, u6)}')
print(f'{c1=}\n{c3=}\n{check_equal_dicts(c1, c3)}')
print(f'{c2=}\n{c4=}\n{check_equal_dicts(c2, c4)}')
print("-------------------")
print(f'{tk_attrs_1=}')
print(f'{tk_attrs_2=}')
print("-------------------")
print(f'{claim_ops_3=}')
print(f'{claim_ops_6=}')
print("-------------------")
print(f'{claim_ops_bf=}')
print(f'{claim_ops_af=}')

input("Closing Chain Simulator. Continue...")

stop_chain_sim_stack(chain_sim)

In [ ]:
user = users_init()[0]
dummy_proxy_contract = DummyProxyContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqrul7un3rkhfktwaa7e2hhvfs6d4kum602jpsw2pwtq")

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
if user_tk_nonce:
    token = ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)
    tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.address, "claimRewardsOnBehalf"])
else:
    user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
    tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)

Enter farm compare

In [ ]:
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
advance_blocks(3)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restart Chain Simulator then press Enter to continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
advance_blocks(1)

farm_upgrade()  # eats 2 blocks

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

u4 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(user)

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u4)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")

input("Closing Chain Simulator; press Enter to continue...")
stop_chain_sim_stack(chain_sim)

Enter farm no consolidation compare

In [ ]:
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
advance_blocks(3)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_no_consolidation_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restart Chain Simulator then press Enter to continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
advance_blocks(1)

farm_upgrade()  # eats 2 blocks

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_no_consolidation_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

u4 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(user)

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u4)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")

input("Closing Chain Simulator; press Enter to continue...")
stop_chain_sim_stack(chain_sim)

In [ ]:
setup_chain_sim()

Enter farm on behalf compare

In [ ]:
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
initial_blocks = 10
advance_blocks(initial_blocks)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restarting Chain Simulator. Continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
permissions_hub_contract = deploy_permissions_hub() # eats 1 block
consumed_blocks += 1

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)
consumed_blocks += 1

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
consumed_blocks += 1


permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)
consumed_blocks += 1

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
farming_tk_balance = get_farming_token_balance_for_account(user.address.bech32())
if user_tk_nonce and farming_tk_balance:
    tokens = [ESDTToken(farm_contract.farmingToken, 0, farming_tk_balance),
              ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)]

    tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [tokens, 0, farm_contract.address, "enterFarmOnBehalf", user.address.bech32()])
else:
    # the farm token is owned by the caller and the farming token is on user
    user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
    token = ESDTToken(farm_contract.farmingToken, 0, farming_tk_balance)
    tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, 
                                                                                                        farm_contract.address, "enterFarmOnBehalf", user.address.bech32()])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)

u5 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u6 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u5)
dict_compare(u4, u6)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")
print(claim_ops_2)
print(claim_ops_4)

input("Closing Chain Simulator; press Enter to continue...")
stop_chain_sim_stack(chain_sim)

Enter on behalf without compounding

In [ ]:
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
initial_blocks = 10
advance_blocks(initial_blocks)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_no_consolidation_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restarting Chain Simulator. Continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
permissions_hub_contract = deploy_permissions_hub() # eats 1 block
consumed_blocks += 1

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)
consumed_blocks += 1

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
consumed_blocks += 1


permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)
consumed_blocks += 1

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

farming_tk_balance = get_farming_token_balance_for_account(user.address.bech32())
if farming_tk_balance:
    tokens = [ESDTToken(farm_contract.farmingToken, 0, farming_tk_balance)]
    tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [tokens, 0, farm_contract.address, "enterFarmOnBehalf", user.address.bech32()])

advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)

u5 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u6 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u5)
dict_compare(u4, u6)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")
print(claim_ops_2)
print(claim_ops_4)

input("Closing Chain Simulator; press Enter to continue...")
stop_chain_sim_stack(chain_sim)

Enter farm on behalf mixed

In [ ]:
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
user2 = users_init()[1]
initial_blocks = 10
advance_blocks(initial_blocks)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

# send user's position away
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
multi_esdt_transfer(context.network_provider.proxy, 10000000, user, WrapperAddress(config.ZERO_CONTRACT_ADDRESS), [ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)])

# send user2's position to user and use this position to enter farm
user_tk_nonce, user_tk_balance = get_position_for_account(user2.address.bech32())
multi_esdt_transfer(context.network_provider.proxy, 10000000, user2, user.address, [ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)])
advance_blocks(1)
sleep(2)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

input("Restarting Chain Simulator. Continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[0]
user2 = users_init()[1]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
permissions_hub_contract = deploy_permissions_hub() # eats 1 block
consumed_blocks += 1

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)
consumed_blocks += 1

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
consumed_blocks += 1


permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)
consumed_blocks += 1

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u32 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

# send user2's position to dummy proxy
user_tk_nonce, user_tk_balance = get_position_for_account(user2.address.bech32())
multi_esdt_transfer(context.network_provider.proxy, 10000000, user2, WrapperAddress(dummy_proxy_contract.address), [ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)])
advance_blocks(1)
sleep(2)

# enter farm on behalf with user's farming position but proxy's farm token
farming_tk_balance = get_farming_token_balance_for_account(user.address.bech32())
# the farm token is owned by the caller and the farming token is on user
proxy_tk_nonce, proxy_tk_balance = get_position_for_account(dummy_proxy_contract.address)
token = ESDTToken(farm_contract.farmingToken, 0, farming_tk_balance)
tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.farmToken, proxy_tk_nonce, proxy_tk_balance, 
                                                                                                        farm_contract.address, "enterFarmOnBehalf", user.address.bech32()])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)

u5 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u6 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))

a, r, m, s = dict_compare(u1, u3)
dict_compare(u2, u5)
dict_compare(u4, u6)
dict_compare(c1, c3)
dict_compare(c2, c4)
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_3)
print("-------------------")
print(claim_ops_2)
print(claim_ops_4)

input("Closing Chain Simulator; press Enter to continue...")
stop_chain_sim_stack(chain_sim)

Freestyle

In [ ]:
user = users_init()[0]
advance_blocks(3)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

In [ ]:
farm_upgrade()  # eats 2 blocks

permissions_hub_contract = deploy_permissions_hub() # eats 1 block

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

In [ ]:
permissions_hub_contract.add_to_blacklist(context.deployer_account, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)

In [ ]:
permissions_hub_contract.remove_from_blacklist(context.deployer_account, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)


In [ ]:
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
if user_tk_nonce:
    token = ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)
    tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.address, "claimRewardsOnBehalf"])
else:
    user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
    tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])

advance_blocks(1)

In [ ]:
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
farming_tk_balance = get_farming_token_balance_for_account(user.address.bech32())
if user_tk_nonce and farming_tk_balance:
    tokens = [ESDTToken(farm_contract.farmingToken, 0, farming_tk_balance),
              ESDTToken(farm_contract.farmToken, user_tk_nonce, user_tk_balance)]

    tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [tokens, 0, farm_contract.address, "enterFarmOnBehalf", user.address.bech32()])
else:
    # the farm token is owned by the caller and the farming token is on user
    user_tk_nonce, user_tk_balance = get_position_for_account(dummy_proxy_contract.address)
    token = ESDTToken(farm_contract.farmingToken, 0, farming_tk_balance)
    tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.farmToken, user_tk_nonce, user_tk_balance, 
                                                                                                        farm_contract.address, "enterFarmOnBehalf", user.address.bech32()])

advance_blocks(1)

In [ ]:
farming_tk_balance = get_farming_token_balance_for_account(user.address.bech32())
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[ESDTToken(farm_contract.farmingToken, 0, farming_tk_balance)], 0, farm_contract.address, "enterFarmOnBehalf", user.address.bech32()])
advance_blocks(1)

In [ ]:
tx_hash = claim_on_behalf_from_user(user)

In [61]:
advance_blocks(1)